## CS431/631 Data Intensive Distributed Analytics
### Winter 2021 - Assignment 3
---

**Please edit this (text) cell to provide your name and UW student ID number!**
* **Name:** Hasnain Syed
* **ID:** 20565033

---
#### Overview
For this assignment, you will be using Python and Spark to perform some graph analysis, using a graph of the Gnutella server network.   In this graph, each node represents a server, and each (directed) edge represents a connection between servers in Gnutella's peer-to-peer network.  The input file for this assignment, `p2p-Gnutella08-adj.txt`, represents the graph as an adjacency list.   Each server (node) is identified by a unique number, and each line in the file gives the adjacency list for a single server.
For example, this line:
> 91	243	1923	2194

gives the adjacency list for server `91`.   It indicates that there are edges from server `91` to servers `243`, `1923`, and `2194`.    According to the Stanford Network Analysis Project, which collected these data, [the graph includes 6301 servers and 20777 edges](http://snap.stanford.edu/data/p2p-Gnutella08.html).

Run the following block to install Spark and download the input file.

In [2]:
!apt-get update -qq > /dev/null
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
!tar xf spark-2.4.7-bin-hadoop2.7.tgz
!pip install -q findspark
!wget -q https://student.cs.uwaterloo.ca/~cs451/content/cs431/p2p-Gnutella08-adj.txt

and then create a `SparkContext`:

In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"

import findspark
findspark.init()

from pyspark import SparkContext, SparkConf

spark_conf = SparkConf()\
  .setAppName("YourTest")\
  .setMaster("local[*]")

sc = SparkContext.getOrCreate(spark_conf)

---
### Important

###### The questions that follow ask you to implement functions whose prototypes are given to you. Do **NOT** change the prototypes of the functions. Do **NOT** write code outside of functions. All necessary code should be included in the function body (except for import statements). You may declare functions inside of the function body. When marking, we will execute your code by calling the functions from an external program, which is why your code cannot rely on statements running outside functions. Please remove any call to the functions that you may have introduced for test purposes before submitting your notebook.

---
#### Question 1  (6/24 marks):

To get warmed up, you should first write Spark code to confirm or determine some basic properties of the Gnutella graph.  Write code in the provided functions that will return answers to the following questions, as specified in each function's documentation:
- How many nodes and edges are there in the graph?  (This should confirm the numbers given above.)
- How many nodes of each outdegree are there? That is, how many nodes have no outgoing edges, how many have one outgoing edge, how many have two outgoing edges, and so on?
- How many nodes of each indegree are there?

You should use Spark to answer these questions.   Do *not* load the entire graph into your Python driver program.

In [23]:
def num_nodes_edges():
    """Returns a tuple (num_nodes, num_edges)"""

    #### Your code for Question 1.1 should go here
    lines = sc.textFile("p2p-Gnutella08-adj.txt")
    num_nodes = lines.count()
    num_edges_1 = lines.map(lambda x: x.split("\t")).filter(lambda x: len(x) > 1)
    num_edges_final = num_edges_1.map(lambda x: x[1:]).flatMap(lambda x: x).count()

    return (num_nodes, num_edges_final)
    
def out_counts():
    """Returns a dictionary where the keys are the outdegrees, and the 
    values are the number of nodes of the corresponding outdegree """

    #### Your code for Question 1.2 should go here
    lines = sc.textFile("p2p-Gnutella08-adj.txt")
    dict_1 = lines.map(lambda x: x.split("\t")).filter(lambda x: len(x) > 1)
    dict_final = dict_1.map(lambda x: (x[0], (len(x[1:])))).collectAsMap()
    
    return dict_final
    
def in_counts():
    """Returns a dictionary where the keys are the indegrees, and the 
    values are the number of nodes of the corresponding indegree """

    #### Your code for Question 1.3 should go here
    lines = sc.textFile("p2p-Gnutella08-adj.txt")
    dict_1 = lines.map(lambda x: x.split("\t")).map(lambda x: x[1:])
    dict_2 = dict_1.flatMap(lambda x: x).map(lambda x: (x,1))
    dict_final = dict_2.reduceByKey(lambda x,y: x+y).collectAsMap()

    return dict_final




In [24]:
#Test calling for Q1

print(num_nodes_edges())
print(out_counts())
print(in_counts())

(6301, 20777)
{'0': 10, '3': 10, '4': 10, '5': 10, '7': 9, '8': 10, '9': 10, '11': 10, '13': 1, '14': 1, '15': 10, '17': 9, '20': 10, '21': 10, '22': 10, '25': 1, '27': 10, '28': 3, '29': 3, '30': 10, '31': 10, '32': 9, '33': 10, '36': 10, '38': 9, '39': 10, '43': 10, '44': 10, '49': 9, '55': 10, '60': 9, '61': 10, '62': 9, '63': 9, '64': 10, '65': 10, '66': 10, '67': 10, '68': 10, '73': 10, '75': 10, '76': 10, '78': 7, '80': 1, '81': 10, '83': 10, '87': 10, '89': 10, '91': 3, '92': 2, '98': 10, '100': 1, '101': 10, '102': 5, '104': 1, '105': 10, '108': 10, '109': 10, '110': 10, '111': 10, '112': 9, '113': 10, '116': 10, '120': 10, '122': 10, '123': 10, '124': 10, '125': 10, '126': 10, '127': 10, '129': 10, '130': 10, '131': 10, '132': 10, '133': 10, '135': 10, '136': 9, '137': 1, '138': 9, '141': 10, '143': 10, '144': 9, '145': 9, '146': 9, '147': 9, '148': 10, '149': 10, '151': 10, '155': 9, '158': 8, '160': 10, '162': 10, '165': 10, '167': 10, '169': 10, '172': 10, '173': 10, '174':

---
Your main objective for this assignment is to perform *single source personalized page rank* over the Gnutella graph.  To get started, you should make sure that you have a clear understanding of ordinary (i.e., non-personalized) page rank.  Read the description of page rank in Section 5.3 of [the course textbook](http://mapreduce.cc/).   Personalized page rank is like ordinary page rank except:
- One node in the graph is designated as the *source* node. Personalized page rank is performed with respect to that source node.
- Personalized page rank is initialized by assigning all probability mass to the source node, and none to the other nodes. In contrast, ordinary page rank is initialized by giving all nodes the same probability mass.
- Whenever personalized page rank makes a random jump, it jumps back to the source node. In contrast, ordinary page rank may jump to any node.
- In personalized page rank, all probability mass lost dangling nodes is put back into the source nodes.  In ordinary page rank, lost mass is distributed evenly over all nodes.

#### Question 2  (10/24 marks):

Your task is to write a Spark program to perform personalized page rank over the Gnutella graph for a specified number of iterations, and of course a specific node. The function you will implement takes three input values:
- source node id (a non-negative integer)
- iteration count (a positive integer)
- random jump factor value (a float between 0 and 1) - This is 1-B as introduced in the lecture.

The function should perform personalized page rank, with respect to the specified source node, over the Gnutella graph, for the specified number of iterations, using Spark.
The output of your function should be a list of the 10 nodes with the highest personalized page rank with respect to the given source. For each of the 10 nodes, return the node's id and page rank value as a tuple. The list returned by the function should therefore look something like this: `[(node_id_1, highest_pagerank_value), ..., (node_id_10, 10th_highest_pagerank_value)]`

In [25]:
def personalized_page_rank(source_node_id, num_iterations, jump_factor):
    """Returns a list of the 10 nodes with the highest page rank value along with their value, as tuples
    [(node_id_1, highest_pagerank_value), ..., (node_id_10, 10th_highest_pagerank_value)]"""

    # your solution to Question 2 here
    def computeScore(links, score):
        if (len(links) == 0):
          yield (str(source_node_id), score)
        else:
          source_score = score*jump_factor
          yield (str(source_node_id), source_score)
          for link in links:
            outter_link_score = ((score)*(1-jump_factor))/len(links)
            yield (link, outter_link_score)

    graph = sc.textFile("p2p-Gnutella08-adj.txt")

    adj_list = graph.map(lambda x: x.split("\t")).map(lambda x: (x[0], x[1:])).cache()
    rank_score = adj_list.map(lambda x: (x[0], 1.0) if x[0] == str(source_node_id) else (x[0], 0))

    for i in range(num_iterations):
      edge_scores = adj_list.join(rank_score).flatMap(lambda x: computeScore(x[1][0], x[1][1]))
      rank_score = edge_scores.reduceByKey(lambda x,y: x+y)

    top10 = rank_score.map(lambda x: (x[1], x[0])).sortByKey(False).map(lambda x: (x[1], x[0])).take(10)

    return top10



In [26]:
#Test calling for q2
personalized_page_rank(2794, 10, 0.05)

[('2794', 0.3654155907233866),
 ('513', 0.036649507904412855),
 ('2001', 0.03659412767617929),
 ('2292', 0.0364616848502554),
 ('2615', 0.03617013461969297),
 ('1509', 0.03616197848386495),
 ('3215', 0.03614608630353985),
 ('1042', 0.03614599491394115),
 ('4306', 0.036144776088395655),
 ('3907', 0.03614475784022655)]

---
#### Question 3  (4/24 marks):

For the previous question, you implemented personalized page rank that ran for a specified number of iterations.  However, it is also common to write iterative algorithms that run until some specified termination condition is reached.
For example, for page rank, suppose the $p_i(x)$ represents the probability mass assigned to node $x$ after the $i$th iteration of the algorithm.  ($p_0(x)$ is the initial probability mass of node $x$.)   We define the change of $x$'s probability mass on the $i$th iteration as $\lvert p_i(x)-p_{i-1}(x) \rvert$.   Then, we can iterate personalized page rank until the maximum (over all nodes) change is less than a specified threshold, i.e, until all nodes' page ranks have converged.

For this question, modify your personalized page rank implementation from Question 2 so that it iterates until the 
maximum node change is less than $\frac{0.5}{N}$, where $N$ represents the number of nodes in the graph.
This version of the function should take only two inputs: the source node id and the random jump factor.

In [27]:
def personalized_page_rank_stopping_criterion(source_node_id, jump_factor):
    """Returns a list of the 10 nodes with the highest page rank value along with their value, as tuples
    [(node_id_1, highest_pagerank_value), ..., (node_id_10, 10th_highest_pagerank_value)]"""

    # your solution to Question 3 here
    def computeScore(links, score):
        if (len(links) == 0):
          yield (str(source_node_id), score)
        else:
          source_score = score*jump_factor
          yield (str(source_node_id), source_score)
          for link in links:
            outter_link_score = ((score)*(1-jump_factor))/len(links)
            yield (link, outter_link_score)
            
    graph = sc.textFile("p2p-Gnutella08-adj.txt")

    adj_list = graph.map(lambda x: x.split("\t")).map(lambda x: (x[0], x[1:])).cache()
    rank_score = adj_list.map(lambda x: (x[0], 1.0) if x[0] == str(source_node_id) else (x[0], 0))

    N = adj_list.count()

    while(True):
      previous_rank = rank_score
      edge_scores = adj_list.join(rank_score).flatMap(lambda x: computeScore(x[1][0], x[1][1])).cache()
      rank_score = edge_scores.reduceByKey(lambda x,y: x+y).cache()
      difference = rank_score.join(previous_rank).map(lambda x: (x[0], (x[1][0],x[1][1]))).map(
                    lambda x: (x[1], x[0])).sortByKey(False).map(lambda x: (x[1], x[0]))
      difference_filtered = difference.filter(lambda x: abs(x[1][0]-x[1][1]) > 0.5/N)

      if (difference_filtered.count() == 0):
        break

    top10 = rank_score.map(lambda x: (x[1], x[0])).sortByKey(False).map(lambda x: (x[1], x[0])).take(10)

    return top10



In [28]:
personalized_page_rank_stopping_criterion(2750, 0.1)

[('2750', 0.3022039487071977),
 ('7', 0.034643205383200135),
 ('174', 0.03360131714530234),
 ('128', 0.02931106394840475),
 ('856', 0.02855446924836467),
 ('717', 0.02845676942362641),
 ('368', 0.02844554779693092),
 ('424', 0.02815377725062892),
 ('125', 0.027985045265408178),
 ('238', 0.027689556656325623)]

---
#### Question 4  (4/24 marks):

Spark provides the ability to *cache* RDDs.   This is useful when an RDD will be used more than once.   Instead of computing the same RDD multiple times, it can be computed once, cached, and then re-used from the cache.   Read about caching in the Spark textbook, or in the [persistence section of the Spark RDD programming guide](https://spark.apache.org/docs/latest/rdd-programming-guide.html#rdd-persistence).   Caching can be particularly effective for iterative Spark applications, like those you are writing for this assignment.

For this question, go back to the code that you wrote to answer Question 3, and add caching.   (Caching will not affect the functionality of your code, i.e., what it computes.   It should only affect performance.)   Don't worry about different persistence levels.   Just use `cache()`, which caches RDDs at the default persistence level.

In addition to adding `cache()` calls in your code, use the text cell below to briefly explain how you decided which RDDs to cache. 

If you were not able to finish Question 3, add caching annotations to your solution for Question 2 instead.

---
#### Your answer to Question 4:

**Line 18:** adj_list = graph.map(lambda x: x.split("\t")).map(lambda x: (x[0], x[1:])).cache()

I added cache on this RDD since this RDD is the data structure that holds all the data about the links of each node. This RDD never changes, since the links between the servers are static throughout, so it's more efficient to cache this in the begginning and use that cached version through every iteration.

**Line 25:** edge_scores = adj_list.join(rank_score).flatMap(lambda x: computeScore(x[1][0], x[1][1])).cache()

I added a cache to this RDD since this is used to calculate rank_score by reducing by keys

**Line 26:** rank_score = edge_scores.reduceByKey(lambda x,y: x+y).cache()

I added cache to this RDD since I save this RDD in previous_rank variable to I can calculate the difference between the probabilities. 




---
Don't forget to save your workbook!   When you are finished and you are ready to submit your assignment, download your notebook file (.ipynb) from the hub to your machine, and then follow the submission instructions in the assignment.